In [6]:
import pandas as pd

# Load dataset
df = pd.read_csv("user_tag_dataset_limited_users.csv")

# Function to predict most used tags for a user
def predict_tags_for_user(user_id, top_k=3):
    user_data = df[df["User_ID"] == user_id]
    
    if user_data.empty:
        return f"❌ No data found for user {user_id}"
    
    # Sum tag usage across all trips
    tag_totals = user_data.drop(columns=["User_ID"]).sum().sort_values(ascending=False)
    
    # Return top_k most frequent tags
    predicted_tags = tag_totals.head(top_k).index.tolist()
    return predicted_tags

# 🔮 Example: Predict for user 105
predicted = predict_tags_for_user(101)
print(f"🔮 Predicted tags for user 101: {predicted}")


🔮 Predicted tags for user 101: ['Mountains', 'Relaxation', 'Nature']


In [7]:
import random
import pandas as pd

# ----------------------------
# Step 1: Define States and Actions
# ----------------------------

user_tags = ["Nature", "Adventure", "Relaxation"]
places = ["Manali", "Goa", "Rishikesh", "Jaipur", "Kerala"]

# Q-table to store values for (user_tag, place)
Q = {(tag, place): 0.0 for tag in user_tags for place in places}

# ----------------------------
# Step 2: Parameters
# ----------------------------

alpha = 0.5        # Learning rate
gamma = 0.8        # Discount factor (not used much in 1-step update)
epsilon = 0.3      # Exploration rate
episodes = 50      # Number of simulated user sessions

# ----------------------------
# Step 3: Simulated User Preferences (to create mock feedback)
# ----------------------------

# User "likes" certain place types based on tag
preferences = {
    "Nature": "Manali",
    "Adventure": "Rishikesh",
    "Relaxation": "Goa"
}

# ----------------------------
# Step 4: Epsilon-Greedy Policy Function
# ----------------------------

def choose_place(user_tag):
    if random.uniform(0, 1) < epsilon:
        # Explore
        return random.choice(places)
    else:
        # Exploit best known
        best = max(places, key=lambda p: Q[(user_tag, p)])
        return best

# ----------------------------
# Step 5: Simulate Learning Over Sessions
# ----------------------------

for episode in range(episodes):
    # Choose a user tag (state)
    tag = random.choice(user_tags)

    # Choose a place using epsilon-greedy
    place = choose_place(tag)

    # Simulate feedback: +1 if preferred, else -1
    reward = 1 if preferences[tag] == place else -1

    # Update Q-value using basic Q-learning update
    old_q = Q[(tag, place)]
    Q[(tag, place)] = old_q + alpha * (reward - old_q)

# ----------------------------
# Step 6: Show Learned Q-values
# ----------------------------

q_table = pd.DataFrame(index=user_tags, columns=places)
for (tag, place), value in Q.items():
    q_table.loc[tag, place] = round(value, 2)

print("📊 Final Q-Table (What the system has learned):\n")
print(q_table)

# ----------------------------
# Step 7: Recommend Function (Demo use case)
# ----------------------------

def recommend(user_tag):
    best_place = max(places, key=lambda p: Q[(user_tag, p)])
    return f"✅ Recommended place for '{user_tag}': {best_place}"

# Example recommendations
print("\n🔮 Recommendations:")
for tag in user_tags:
    print(recommend(tag))


📊 Final Q-Table (What the system has learned):

           Manali   Goa Rishikesh Jaipur Kerala
Nature        1.0 -0.75      -0.5  -0.75    0.0
Adventure    -0.5  -0.5      0.97  -0.75  -0.75
Relaxation  -0.88   1.0     -0.75   -0.5  -0.75

🔮 Recommendations:
✅ Recommended place for 'Nature': Manali
✅ Recommended place for 'Adventure': Rishikesh
✅ Recommended place for 'Relaxation': Goa


In [8]:
import pandas as pd
import random

# Define simulation parameters
budgets = ["low", "medium", "high"]
trip_tags = ["Adventure", "Relaxation", "Nature", "Cultural"]
places = ["Manali", "Goa", "Rishikesh", "Jaipur", "Kerala", "Ladakh", "Pondicherry"]

# Simulate 200 user interactions
data = []
for _ in range(200):
    user_id = random.randint(1, 10)
    budget = random.choice(budgets)
    tag = random.choice(trip_tags)
    place = random.choice(places)

    # Logic: simulate if user accepts based on matching
    accepted = 1 if (
        (tag == "Adventure" and place in ["Rishikesh", "Ladakh"]) or
        (tag == "Relaxation" and place in ["Goa", "Pondicherry"]) or
        (tag == "Nature" and place in ["Manali", "Kerala"]) or
        (tag == "Cultural" and place in ["Jaipur", "Kerala"])
    ) else 0

    data.append([user_id, budget, tag, place, accepted])

df = pd.DataFrame(data, columns=["user_id", "budget", "tag", "place", "accepted"])
df.to_csv("travel_demo_dataset.csv", index=False)
df.head()

,user_id,budget,tag,place,accepted
0,5,low,Cultural,Manali,0
1,4,low,Adventure,Kerala,0
2,6,low,Nature,Ladakh,0
3,4,medium,Adventure,Rishikesh,1
4,5,low,Nature,Ladakh,0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("travel_demo_dataset.csv")

# Features and target
X = df[["budget", "tag", "place"]]
y = df["accepted"]

# One-hot encode categorical features
categorical_features = ["budget", "tag", "place"]
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(), categorical_features)]
)

# Build pipeline
model = Pipeline([
    ("preprocess", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.93      1.00      0.96        27
           1       1.00      0.85      0.92        13

    accuracy                           0.95        40
   macro avg       0.97      0.92      0.94        40
weighted avg       0.95      0.95      0.95        40



In [ ]:
# Example user inputs
user_input = pd.DataFrame([{
    "budget": "medium",
    "tag": "Adventure",
    "place": "Rishikesh"
}])

prediction = model.predict(user_input)
print("✅ User likely to accept!" if prediction[0] == 1 else "❌ User likely to reject.")

prediction: 1
✅ User likely to accept!
